In [1]:
from tinyagents import chainable, respond

@chainable
class Agent:
    name: str = "main_agent"

    def __init__(self):
        pass

    def run(self, input: str):
        return "Hello! I was provided with some tool outputs: " + str(input)
    
    def output_handler(self, output):
        return respond(output)

@chainable
class Tool1:
    """ Some tool """
    name: str = "tool1"

    def run(self, input):
        return "Tool 1 has been triggered."

@chainable
class Tool2:
    """ Some other tool """
    def __init__(self, phrase: str):
        self.phrase = phrase

    def run(self, input):
        return f"Tool 2 has been {self.phrase}."

/Users/adamhedib/Library/Caches/pypoetry/virtualenvs/tinyagents-JQO3J91T-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-16 21:24:59,734	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
branch1 = Tool1() / Tool2(phrase="executed")

def router(x):
    if x.startswith("blue"):
        return Tool1.name
    
    return Tool2.name
    

branch1.bind_router(router)

ConditionalBranch(tool1 | Tool2)

In [3]:
graph = branch1 | Agent()
str(graph)

'ConditionalBranch(tool1 | Tool2) -> main_agent'

In [4]:
# Run without using Ray
runner = graph.compile()
runner.invoke("blue!")


 > Running node: ConditionalBranch(tool1 | Tool2)

	Input: blue!

	Output (ConditionalBranch(tool1 | Tool2)): {
  "content": "Tool 1 has been triggered.",
  "action": null,
  "ref": null
}

 > Running node: main_agent

	Input: Tool 1 has been triggered.

	Output (main_agent): {
  "content": "Hello! I was provided with some tool outputs: Tool 1 has been triggered.",
  "action": "respond",
  "ref": null
}


'Hello! I was provided with some tool outputs: Tool 1 has been triggered.'

In [5]:
runner.invoke("yellow!")


 > Running node: ConditionalBranch(tool1 | Tool2)

	Input: yellow!

	Output (ConditionalBranch(tool1 | Tool2)): {
  "content": "Tool 2 has been executed.",
  "action": null,
  "ref": null
}

 > Running node: main_agent

	Input: Tool 2 has been executed.

	Output (main_agent): {
  "content": "Hello! I was provided with some tool outputs: Tool 2 has been executed.",
  "action": "respond",
  "ref": null
}


'Hello! I was provided with some tool outputs: Tool 2 has been executed.'

In [6]:
# Run using Ray
runner = graph.compile(
    use_ray=True,
    ray_options={
        "tool1": {
            "num_replicas": 3
        },
        "tool2": {
            "num_replicas": 3,
            # "placement_group_strategy": ... 
        },
        "runner": {
            "num_replicas": 2
        }
    }
)
runner

In [ ]:
from ray import serve

# we can now serve it using Ray Serve

app = serve.run(runner, name="MyApp")

2024-07-16 21:25:01,849	INFO worker.py:1762 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8266 
(ProxyActor pid=91494) INFO 2024-07-16 21:25:03,789 proxy 127.0.0.1 proxy.py:1179 - Proxy starting on node 9e82adfd793582eb905bc69a17b0d83907df45db331c3a8cd5bc53a9 (HTTP port: 8000).
2024-07-16 21:25:03,815	INFO handle.py:126 -- Created DeploymentHandle '0n6kt5xc' for Deployment(name='tool1', app='MyApp').
2024-07-16 21:25:03,816	INFO handle.py:126 -- Created DeploymentHandle 'nfon0gsb' for Deployment(name='Tool2', app='MyApp').
2024-07-16 21:25:03,816	INFO handle.py:126 -- Created DeploymentHandle '6i6we0zk' for Deployment(name='main_agent', app='MyApp').
2024-07-16 21:25:03,817	INFO handle.py:126 -- Created DeploymentHandle 'fbtkerzx' for Deployment(name='tool1', app='MyApp').
2024-07-16 21:25:03,817	INFO handle.py:126 -- Created DeploymentHandle 'tbs7qmls' for Deployment(name='Tool2', app='MyApp').
2024-07-16 21:25:03,817	INFO handle.py:126 -- Created DeploymentH

(ServeReplica:MyApp:main_agent pid=91499) 
(ServeReplica:MyApp:main_agent pid=91499)  > Running node: main_agent
(ServeReplica:MyApp:main_agent pid=91499) 
(ServeReplica:MyApp:main_agent pid=91499) 	Input: Tool 1 has been triggered.
(ServeReplica:MyApp:main_agent pid=91499) 
(ServeReplica:MyApp:main_agent pid=91499) 	Output (main_agent): {
(ServeReplica:MyApp:main_agent pid=91499)   "content": "Hello! I was provided with some tool outputs: Tool 1 has been triggered.",
(ServeReplica:MyApp:main_agent pid=91499)   "action": "respond",
(ServeReplica:MyApp:main_agent pid=91499)   "ref": null
(ServeReplica:MyApp:main_agent pid=91499) }
(ServeReplica:MyApp:main_agent pid=91499) 
(ServeReplica:MyApp:main_agent pid=91499)  > Running node: main_agent
(ServeReplica:MyApp:main_agent pid=91499) 
(ServeReplica:MyApp:main_agent pid=91499) 	Input: Tool 1 has been triggered.
(ServeReplica:MyApp:main_agent pid=91499) 
(ServeReplica:MyApp:main_agent pid=91499) 	Output (main_agent): {
(ServeReplica:MyApp:

(ServeReplica:MyApp:main_agent pid=91499) INFO 2024-07-16 21:25:06,874 MyApp_main_agent w51irvql 0d4c2159-2ec3-4cba-8a0f-c8bf1d726e1e replica.py:373 - AINVOKE OK 1.3ms
(ServeReplica:MyApp:main_agent pid=91499) INFO 2024-07-16 21:25:06,907 MyApp_main_agent w51irvql afc3e705-69cb-479f-8fbf-53f618940545 /invoke replica.py:373 - AINVOKE OK 0.6ms
(ServeReplica:MyApp:tool1 pid=91496) INFO 2024-07-16 21:25:06,905 MyApp_tool1 847ttylr afc3e705-69cb-479f-8fbf-53f618940545 /invoke replica.py:373 - INVOKE OK 1.0ms
(ServeReplica:MyApp:tool1 pid=91495) INFO 2024-07-16 21:25:06,868 MyApp_tool1 fk5u5lfc 0d4c2159-2ec3-4cba-8a0f-c8bf1d726e1e replica.py:373 - INVOKE OK 0.9ms
(ServeReplica:MyApp:runner pid=91500) INFO 2024-07-16 21:25:06,875 MyApp_runner vco8sbup 0d4c2159-2ec3-4cba-8a0f-c8bf1d726e1e replica.py:373 - AINVOKE OK 17.0ms
(ServeReplica:MyApp:runner pid=91500) INFO 2024-07-16 21:25:06,907 MyApp_runner vco8sbup afc3e705-69cb-479f-8fbf-53f618940545 /invoke replica.py:373 - __CALL__ OK 6.7ms


(ServeReplica:MyApp:main_agent pid=91499) INFO 2024-07-16 21:25:17,001 MyApp_main_agent w51irvql 50d3456f-ccaf-4227-b807-31ecc90508ea /invoke replica.py:373 - AINVOKE OK 1.8ms
(ServeReplica:MyApp:runner pid=91501) INFO 2024-07-16 21:25:17,001 MyApp_runner 66wygpf1 50d3456f-ccaf-4227-b807-31ecc90508ea /invoke replica.py:373 - __CALL__ OK 63.3ms
(ServeReplica:MyApp:tool1 pid=91496) INFO 2024-07-16 21:25:16,989 MyApp_tool1 847ttylr 50d3456f-ccaf-4227-b807-31ecc90508ea /invoke replica.py:373 - INVOKE OK 6.5ms


(ServeReplica:MyApp:main_agent pid=91499) INFO 2024-07-16 21:25:21,102 MyApp_main_agent w51irvql 471b19fb-0a25-4d68-b35d-493172fe15ef replica.py:373 - AINVOKE OK 0.7ms
(ServeReplica:MyApp:tool1 pid=91496) INFO 2024-07-16 21:25:21,097 MyApp_tool1 847ttylr 471b19fb-0a25-4d68-b35d-493172fe15ef replica.py:373 - INVOKE OK 0.9ms
(ServeReplica:MyApp:runner pid=91500) INFO 2024-07-16 21:25:21,103 MyApp_runner vco8sbup 471b19fb-0a25-4d68-b35d-493172fe15ef replica.py:373 - AINVOKE OK 80.6ms


(ServeReplica:MyApp:main_agent pid=91499) 
(ServeReplica:MyApp:main_agent pid=91499)  > Running node: main_agent
(ServeReplica:MyApp:main_agent pid=91499) 
(ServeReplica:MyApp:main_agent pid=91499) 	Input: Tool 1 has been triggered.
(ServeReplica:MyApp:main_agent pid=91499) 
(ServeReplica:MyApp:main_agent pid=91499) 	Output (main_agent): {
(ServeReplica:MyApp:main_agent pid=91499)   "content": "Hello! I was provided with some tool outputs: Tool 1 has been triggered.",
(ServeReplica:MyApp:main_agent pid=91499)   "action": "respond",
(ServeReplica:MyApp:main_agent pid=91499)   "ref": null
(ServeReplica:MyApp:main_agent pid=91499) }
(ServeReplica:MyApp:runner pid=91500) 	Output (ConditionalBranch(tool1 | Tool2)): {
(ServeReplica:MyApp:runner pid=91500)   "content": "Tool 1 has been triggered.",
(ServeReplica:MyApp:runner pid=91500)   "action": null,
(ServeReplica:MyApp:runner pid=91500) }


(ServeReplica:MyApp:main_agent pid=91499) INFO 2024-07-16 21:25:38,421 MyApp_main_agent w51irvql 381ebe23-47f4-4058-a2cd-42abcc1e4957 replica.py:373 - AINVOKE OK 1.7ms
(ServeReplica:MyApp:runner pid=91501) INFO 2024-07-16 21:25:38,421 MyApp_runner 66wygpf1 381ebe23-47f4-4058-a2cd-42abcc1e4957 replica.py:373 - AINVOKE OK 25.5ms
(ServeReplica:MyApp:tool1 pid=91497) INFO 2024-07-16 21:25:38,416 MyApp_tool1 g01z60ub 381ebe23-47f4-4058-a2cd-42abcc1e4957 replica.py:373 - INVOKE OK 2.4ms


(ServeReplica:MyApp:runner pid=91501) 
(ServeReplica:MyApp:runner pid=91501)  > Running node: ConditionalBranch(tool1 | Tool2)
(ServeReplica:MyApp:runner pid=91501) 
(ServeReplica:MyApp:runner pid=91501) 	Input: blue!
(ServeReplica:MyApp:runner pid=91501) 
(ServeReplica:MyApp:runner pid=91501) 	Output (ConditionalBranch(tool1 | Tool2)): {
(ServeReplica:MyApp:runner pid=91501)   "content": "Tool 1 has been triggered.",
(ServeReplica:MyApp:runner pid=91501)   "action": null,
(ServeReplica:MyApp:runner pid=91501)   "ref": null
(ServeReplica:MyApp:runner pid=91501) }
(ServeReplica:MyApp:main_agent pid=91499) 
(ServeReplica:MyApp:main_agent pid=91499)  > Running node: main_agent
(ServeReplica:MyApp:main_agent pid=91499) 
(ServeReplica:MyApp:main_agent pid=91499) 	Input: Tool 1 has been triggered.
(ServeReplica:MyApp:main_agent pid=91499) 
(ServeReplica:MyApp:main_agent pid=91499) 	Output (main_agent): {
(ServeReplica:MyApp:main_agent pid=91499)   "content": "Hello! I was provided with some 

(ServeReplica:MyApp:main_agent pid=91499) 
(ServeReplica:MyApp:main_agent pid=91499)  > Running node: main_agent
(ServeReplica:MyApp:main_agent pid=91499) 
(ServeReplica:MyApp:main_agent pid=91499) 	Input: Tool 1 has been triggered.
(ServeReplica:MyApp:main_agent pid=91499) 
(ServeReplica:MyApp:main_agent pid=91499) 	Output (main_agent): {
(ServeReplica:MyApp:main_agent pid=91499)   "content": "Hello! I was provided with some tool outputs: Tool 1 has been triggered.",
(ServeReplica:MyApp:main_agent pid=91499)   "action": "respond",
(ServeReplica:MyApp:main_agent pid=91499)   "ref": null
(ServeReplica:MyApp:main_agent pid=91499) }


In [8]:
!serve status

proxies:
  6f0d1b1ef00348e85c1f0ddae8575304cb220b0be8ab9e1830a73ea7: HEALTHY
applications:
  default:
    status: RUNNING
    message: ''
    last_deployed_time_s: 1721160715.955894
    deployments:
      legal_researcher:
        status: HEALTHY
        status_trigger: CONFIG_UPDATE_COMPLETED
        replica_states:
          RUNNING: 1
        message: ''
      search_tool:
        status: HEALTHY
        status_trigger: CONFIG_UPDATE_COMPLETED
        replica_states:
          RUNNING: 1
        message: ''
      runner:
        status: HEALTHY
        status_trigger: CONFIG_UPDATE_COMPLETED
        replica_states:
          RUNNING: 1
        message: ''
target_capacity: null


In [13]:
result = await app.ainvoke.remote("blue!")
result

2024-07-16 21:25:38,390	INFO handle.py:126 -- Created DeploymentHandle 'r2x2vae9' for Deployment(name='runner', app='MyApp').


NodeOutput(content='Hello! I was provided with some tool outputs: Tool 1 has been triggered.', action='respond', ref=None)

In [14]:
# we can also interact with the application via REST

import requests

response = requests.post("http://localhost:8000/invoke", data="blue")

print(response.text)

{"content":"Hello! I was provided with some tool outputs: Tool 1 has been triggered.","action":"respond","ref":null}


(ServeReplica:MyApp:tool1 pid=91496) INFO 2024-07-16 21:25:47,913 MyApp_tool1 847ttylr b1d229ab-9b55-4cdc-9c0f-f62096462904 /invoke replica.py:373 - INVOKE OK 7.4ms
(ServeReplica:MyApp:main_agent pid=91499) INFO 2024-07-16 21:25:47,946 MyApp_main_agent w51irvql b1d229ab-9b55-4cdc-9c0f-f62096462904 /invoke replica.py:373 - AINVOKE OK 4.8ms
(ServeReplica:MyApp:runner pid=91500) INFO 2024-07-16 21:25:47,951 MyApp_runner vco8sbup b1d229ab-9b55-4cdc-9c0f-f62096462904 /invoke replica.py:373 - __CALL__ OK 58.0ms
